In [1]:
import sys
sys.path.append('../')

from utils import save_data_with_pickle, load_data_with_pickle
from CorpusManager import CorpusManager


In [2]:
CORPUS_PATH = '/datahdd/vmanuel/ELMo/Corpora/shuffled_text_with_words'

c = CorpusManager()
c.read_corpus(CORPUS_PATH, 100000)

read input corpus


100%|██████████| 100000/100000 [00:04<00:00, 23031.71it/s]


In [3]:
PICKLE_PATH = ('../../../source_files/pickles/')

found_entity_dict = load_data_with_pickle(PICKLE_PATH + 'found_entity_dict_static')


In [4]:
found_entity_dict

{'Name': {'commissar'},
 'Winery': {'cellars', 'corner'},
 'School': {'abbey',
  'anniversary',
  'fare',
  'friday',
  'holiday',
  'monday',
  'pompeii',
  'saturday',
  'shabbat',
  'sunday',
  'thursday',
  'tuesday'},
 'GolfTournament': set(),
 'Bridge': {'childhood', 'era', 'footbridge', 'precambrian', 'viaduct'},
 'IceHockeyLeague': set(),
 'MilitaryStructure': {'ansps',
  'blestium',
  'blockhouse',
  'bunker',
  'gariannonum',
  'glevum'},
 'HistoricBuilding': set(),
 'Work': {'aes',
  'asset',
  'buyout',
  'career',
  'caricature',
  'chronology',
  'cinematography',
  'conjecture',
  'crystallography',
  'demon',
  'distinct',
  'ecology',
  'economics',
  'electronics',
  'empiricism',
  'fell',
  'forestry',
  'geology',
  'grape',
  'hadith',
  'interview',
  'ipv',
  'jurisprudence',
  'justice',
  'kinship',
  'landmark',
  'martyrology',
  'medicine',
  'palimpsest',
  'photography',
  'physics',
  'pilgrimage',
  'reputation',
  'revenant',
  'ridge',
  'rolling',
  

In [5]:
c.joined_corpus[51634]

'myrmecocystus mexicanus species ant genus myrmecocystus one six genera bear common name '

In [6]:
word_indexes = load_data_with_pickle(PICKLE_PATH + 'word_occurrence_index_32150_100000static')

In [7]:
word_indexes

{'myrmecocystus': [(51634, [0, 5])],
 'hobart': [(2276, [55]),
  (3659, [9]),
  (4934, [20]),
  (9227, [43]),
  (9988, [17]),
  (11850, [41]),
  (14299, [13]),
  (14770, [19]),
  (17603, [102, 156]),
  (18571, [14]),
  (18745, [16]),
  (22119, [106, 116]),
  (22619, [79, 138]),
  (25673, [21]),
  (28355, [49, 56]),
  (28468, [21]),
  (31155, [13]),
  (32338, [2]),
  (32816, [13]),
  (37378, [57]),
  (37841, [9]),
  (40644, [1, 24]),
  (41965, [75]),
  (42039, [11, 137]),
  (42720, [5]),
  (46571, [233, 237, 356]),
  (48625, [154]),
  (48995, [14]),
  (49801, [11]),
  (53645, [56]),
  (54962, [1]),
  (62651, [24, 43, 67, 136]),
  (63462, [49]),
  (63966, [83, 90, 146]),
  (64298, [0, 10]),
  (66004, [8]),
  (68081, [8, 54]),
  (71625, [11]),
  (74137, [109]),
  (75460, [23]),
  (78321, [4]),
  (85566, [19]),
  (86378, [76]),
  (86410, [351, 800]),
  (87423, [20]),
  (87562, [32, 64]),
  (89902, [7, 11]),
  (92811, [105, 197, 246, 251, 264, 278]),
  (95293, [11, 15]),
  (95821, [17]),
  

In [8]:
from collections import defaultdict
reverse_dict = defaultdict(list)

entity_dict = load_data_with_pickle(PICKLE_PATH + 'entity_dict')

for k, words in entity_dict.items():
    for w in words:
        reverse_dict[w].append(k)

In [9]:
all_entities = []

for k, v in entity_dict.items():
    all_entities.extend(v)
all_entities = set(all_entities)

In [10]:
from colorama import Fore, Back, Style

In [11]:
def check_entity_in_row(ENTITY, ROW, verbose):
    
    a = extract_rows_occurrency(ENTITY.split(' '), ROW)
    if verbose:
        print('row: {}'.format(ROW))
        print('entity: {}'.format(ENTITY))
        print('indexes of each word: {}'.format([(w, o) for w, o in zip(ENTITY.split(' '), a)]))
        print(' '.join([Fore.BLACK + c_w if c_w not in ENTITY.split(' ') else Fore.RED + c_w for c_w in c.corpus[ROW]]))
        print(Fore.BLACK + '--------')
    values = []
    for value in a[0]:
        occ = [(value, 0)]
        i = 1
        while i < len(a) and value + i in a[i]:
            occ.append(((value + i), i))
            i += 1
        if i == len(a):
            if verbose:
                print(Fore.BLACK + 'entity present in this sentence at the index {}'.format(value))
            values.append(value)
        else:
            if verbose:
                print(Fore.BLACK + 'entity not present in this sentence at the index {}'.format(value))
        if verbose:
            print(Fore.BLACK + '--------')
    if values:
        return values

In [12]:
from tqdm import tqdm
keys = set(word_indexes.keys())

def extract_rows(word):
    return [t[0] for t in word_indexes[word]] #extract the row index from each tuple

def extract_rows_occurrency(word_phrase, row):
    return [t[1] for s in word_phrase for t in word_indexes[s] if t[0] == row]

j = 0
verbose = False
found = defaultdict(list)
for entity in all_entities:
    splitted = entity.split(' ')
    if set(splitted) <= keys:
        # all words in this entity are in the corpus
        i = 0
        rows = extract_rows(splitted[i])
        while rows and i + 1 < len(splitted):
            i += 1
            rows = [r for r in rows if r in extract_rows(splitted[i])]
        if rows and len(splitted) > 1:
            for r in rows:
                b = check_entity_in_row(ENTITY=entity, ROW=r, verbose = verbose)
            if b:
                if verbose:
                    print('entity found at index(es): {}'.format(' '.join([str(v) for v in b])))
                found[entity].append((r, b))
            j +=1
        elif rows:
            pass
print(found)

KeyboardInterrupt: 

In [13]:
all_occurrences = load_data_with_pickle(PICKLE_PATH + 'occurrences_of_entities')
all_occurrences = [(k , v) if type(v[0]) == tuple else (k, v[0]) for k, v in all_occurrences.items() if len(k) > 2]
all_occurrences = {x[0]: x[1] for x in all_occurrences}

In [14]:
all_occurrences

{'slasher film': [(86081, [9])],
 'chamunda': [(6482, [100]), (82558, [40])],
 'reynold': [(32284, [14]), (97522, [41])],
 'finalization': [(64789, [45])],
 'publican': [(94918, [33])],
 'madusa': [(1440, [18])],
 'lorraine franconian': [(22335, [1])],
 'southington connecticut': [(61577, [9])],
 'adem': [(22156, [60]), (38337, [47]), (92827, [42])],
 'lishana deni': [(11498, [46])],
 'thriller genre': [(72150, [47])],
 'pink si': [(1724, [92])],
 'rinca': [(83305, [39])],
 'longman': [(6893, [65]), (52799, [27]), (55889, [14]), (73671, [31])],
 'pinback': [(9813, [12]), (30687, [3, 8, 14])],
 'wrench': [(36881, [16]), (85709, [0, 27]), (87731, [46])],
 'cartoonist': [(1134, [6]),
  (2733, [4]),
  (4801, [8]),
  (5535, [19]),
  (6211, [48]),
  (7876, [19]),
  (8933, [5]),
  (9233, [14]),
  (9664, [7]),
  (16486, [5, 26]),
  (18734, [5]),
  (22374, [8]),
  (26892, [9, 45]),
  (29447, [3]),
  (31255, [9]),
  (31347, [4]),
  (33455, [5]),
  (33701, [7, 48]),
  (33894, [23]),
  (34564, [12

In [15]:
sentences_to_embed = [v[0] for values in all_occurrences.values() for v in values]

print('total found entity mentions: {}'.format(len(sentences_to_embed)))
print('fraction of sentences with entity mentions: {:.2f} ({} on {})'.format(len(set(sentences_to_embed))/len(c.joined_corpus),
                                                                             len(set(sentences_to_embed)),
                                                                             len(c.joined_corpus)))
print('{:.2f} average entity mentions per sentence'.format(len(sentences_to_embed)/len(set(sentences_to_embed))))

total found entity mentions: 331585
fraction of sentences with entity mentions: 0.84 (83880 on 100000)
3.95 average entity mentions per sentence


In [71]:
all_occurrences['the']

[(0, [35, 98]),
 (2, [22]),
 (12, [0, 14]),
 (16, [0, 8]),
 (18, [0, 39]),
 (19, [9, 22, 27]),
 (22, [42, 59]),
 (30, [0, 50, 89, 106, 112, 122]),
 (31, [4, 39, 62, 76, 88, 115]),
 (38, [23]),
 (39, [77]),
 (40, [0, 29]),
 (45, [0, 17, 42, 63]),
 (47, [0, 11, 34, 45, 59]),
 (57, [25]),
 (58, [9, 17]),
 (60, [18]),
 (61, [9]),
 (73, [40]),
 (78, [15]),
 (80, [0, 37]),
 (88, [55, 75]),
 (89, [0, 17]),
 (90, [16]),
 (91, [4, 23, 31]),
 (93, [142, 231]),
 (94, [0, 19]),
 (95, [31, 59]),
 (97, [13]),
 (101, [9]),
 (106, [25]),
 (107, [84, 86, 105, 134, 171, 218, 261, 272]),
 (112, [0, 13]),
 (113, [63]),
 (115, [0, 27, 41]),
 (119, [0, 11]),
 (120, [35, 68]),
 (122, [9]),
 (130, [17, 63, 80]),
 (131, [25]),
 (134, [34, 66]),
 (138, [8]),
 (140, [13, 29]),
 (150, [15]),
 (153, [9]),
 (155, [12, 20, 27, 51, 76, 81]),
 (156, [96, 115]),
 (157, [54]),
 (159, [0, 26]),
 (164, [9]),
 (165, [20]),
 (166, [0, 58, 80, 104, 122, 149]),
 (167, [14, 60, 102]),
 (168, [0, 32]),
 (170, [10, 26, 34]),
 (1

In [24]:
embedding_data_structure = {index:[] for index in sentences_to_embed if index < 10000}

for entity_mention, occurrences in all_occurrences.items():
    for couple in occurrences:
        if couple[0] < 10000:
            embedding_data_structure[couple[0]].append((couple[1], entity_mention))
                    
from collections import OrderedDict


ordered_embedding_data_structure = OrderedDict(sorted(embedding_data_structure.items()))
ordered_embedding_data_structure

OrderedDict([(0,
              [([35, 98], 'the'),
               ([3], 'august'),
               ([105], 'well'),
               ([36], 'new'),
               ([71], 'south'),
               ([17], 'time')]),
             (1, [([20], 'head'), ([22], 'kanpur'), ([7], 'system')]),
             (2,
              [([30], 'skateboarding'),
               ([22], 'the'),
               ([6, 21], 'hill'),
               ([29], 'snowboarding')]),
             (3, [([3], 'system')]),
             (4, [([11, 19], 'sister'), ([54], 'tower'), ([4], 'august')]),
             (7, [([4], 'seychelles')]),
             (8,
              [([24], 'million'),
               ([12], 'bridge'),
               ([57, 64], 'project'),
               ([62], 'seed'),
               ([26], 'live'),
               ([8], 'relief'),
               ([3, 75], 'andrew'),
               ([17, 83, 97], 'food'),
               ([95], 'country')]),
             (10, [([4], 'august')]),
             (11, [([6], 'murder')]),


In [19]:
from allennlp.commands.elmo import ElmoEmbedder

elmo = ElmoEmbedder(cuda_device = 0)

03/05/2020 16:30:49 - INFO - allennlp.commands.elmo -   Initializing ELMo.


In [64]:
ordered_embedding_data_structure

OrderedDict([(0,
              [([35, 98], 'the'),
               ([3], 'august'),
               ([105], 'well'),
               ([36], 'new'),
               ([71], 'south'),
               ([17], 'time')]),
             (1, [([20], 'head'), ([22], 'kanpur'), ([7], 'system')]),
             (2,
              [([30], 'skateboarding'),
               ([22], 'the'),
               ([6, 21], 'hill'),
               ([29], 'snowboarding')]),
             (3, [([3], 'system')]),
             (4, [([11, 19], 'sister'), ([54], 'tower'), ([4], 'august')]),
             (7, [([4], 'seychelles')]),
             (8,
              [([24], 'million'),
               ([12], 'bridge'),
               ([57, 64], 'project'),
               ([62], 'seed'),
               ([26], 'live'),
               ([8], 'relief'),
               ([3, 75], 'andrew'),
               ([17, 83, 97], 'food'),
               ([95], 'country')]),
             (10, [([4], 'august')]),
             (11, [([6], 'murder')]),


In [44]:
vectors_dict = defaultdict(list)

for row_index, occurrences in tqdm(ordered_embedding_data_structure.items()):
    vectors = elmo.embed_sentence(c.corpus[row_index])[2]
    for occ in occurrences:
        for word_index in occ[0]:
            if len(occ[1].split(' ')) == 1:
                vectors_dict[occ[1]].append(vectors[word_index])
            else:
                vecs = [vectors[w_i] for w_i in range(word_index, word_index + len(occ[1].split(' ')))]
                vectors_dict[occ[1]].append(np.mean(vecs, axis = 0))                        
# print('{}'.format('\n'.join(['{}'.format((k, len(v))) for k, v in vectors_dict.items()])))

100%|██████████| 8450/8450 [12:38<00:00, 11.14it/s]


In [45]:
found_entities = set(vectors_dict.keys())
found_entity_dict = {k: set(v).intersection(found_entities) for k,v in entity_dict.items() if set(v).intersection(found_entities)}    

In [46]:
found_entity_dict

{'Name': {'adrian',
  'alessandro',
  'alexandra',
  'alexius',
  'amy',
  'andrew',
  'andrius',
  'anne',
  'artemy',
  'bagrat',
  'beata',
  'benny',
  'blake',
  'bonnie',
  'brenton',
  'broderick',
  'celine',
  'christopher',
  'claudio',
  'cyril',
  'damian',
  'diego',
  'dobroslav',
  'dong',
  'edwin',
  'eero',
  'elias',
  'ellen',
  'eric',
  'etienne',
  'filipe',
  'francesca',
  'gabe',
  'gennady',
  'gerald',
  'gert',
  'ghai',
  'gwendolyn',
  'hamish',
  'helmut',
  'hussein',
  'ignacio',
  'ignatius',
  'ilya',
  'isidore',
  'ivo',
  'jacinto',
  'jarle',
  'joanna',
  'joanne',
  'jodie',
  'johanna',
  'johannes',
  'jon',
  'jorge',
  'joseph',
  'juana',
  'juliana',
  'kalle',
  'kaoru',
  'karim',
  'karla',
  'katherine',
  'kathryn',
  'kathy',
  'kay',
  'kevin',
  'kirara',
  'kirsten',
  'kurt',
  'larry',
  'lars',
  'laurence',
  'lonnie',
  'lucie',
  'lucy',
  'ludvig',
  'luis',
  'mamdouh',
  'martins',
  'masaru',
  'matteo',
  'maximilian',

In [49]:
G = load_data_with_pickle(PICKLE_PATH + 'graph')

found_entity_dict = c.avoid_multilabeling(found_entity_dict, G, file = 'avoid_multilabeling.txt')

Multilabel alerts: 1380
Multilabel alerts: 0


In [72]:
found_entity_dict

{'Name': set(),
 'Swimmer': {'michael wenden'},
 'Tower': set(),
 'School': {'friday', 'monday', 'pompeii', 'sunday'},
 'Bridge': {'bifrost',
  'footbridge',
  'lafranconi bridge',
  'pmp floating bridge',
  'sleeping porch'},
 'IceHockeyLeague': set(),
 'Skater': set(),
 'MilitaryStructure': set(),
 'HistoricBuilding': {'broome park'},
 'Work': {'agronomy',
  'electronics',
  'empiricism',
  'geology',
  'grammatical mood',
  'jurisprudence',
  'martyrology',
  'medicine',
  'nutrition',
  'palimpsest',
  'tandoor'},
 'SoccerClub': {'brooks', 'kingston rowing club', 'skeid'},
 'ArtificialSatellite': {'sacred'},
 'Lake': {'flaten'},
 'Planet': {'eternia', 'negatives', 'thersander'},
 'RouteOfTransportation': set(),
 'Musical': {'mozart'},
 'Manga': {'replay'},
 'WineRegion': {'echezeaux', 'lord'},
 'Writer': {'e l doctorow'},
 'SportFacility': set(),
 'Motorcycle': {'indian', 'superbike', 'tech'},
 'TelevisionStation': {'acts',
  'dusk',
  'heruka',
  'inactive',
  'pink si',
  'wach'}

In [73]:
reverse_dict = defaultdict(list)

# entity_dict = load_data_with_pickle(PICKLE_PATH + 'entity_dict')

for k, words in found_entity_dict.items():
    for w in words:
        reverse_dict[w].append(k)

In [74]:
len(reverse_dict)

1406

In [75]:
X = []
Y = []
entities = []

for label, label_vectors in vectors_dict.items():
    if label in reverse_dict:
        for v in label_vectors:
            X.append(v)
            Y.append(reverse_dict[label])
            entities.append(label)

In [85]:
len(X)

14912

In [86]:
len(Y)

14912

In [87]:
len(entities)

14912

In [93]:
save_data_with_pickle(PICKLE_PATH + '../vectors/elmo_vectors', X)
save_data_with_pickle(PICKLE_PATH + '../vectors/labels', Y)
save_data_with_pickle(PICKLE_PATH + '../vectors/entities', entities)